In [1]:
#Setup

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob, pickle, joblib, os, uproot
from tqdm import tqdm_notebook

import rawio.IO as io
from scdmsPyTools.Traces.Filtering import *
from scdmsPyTools.Traces.Noise import *
from scdmsPyTools.TES.Templates import *
from scdmsPyTools.Traces.Stats import *
#from scdmsPyTools.BatTools.IO import *
rawdir = "/data/midas/"
procdir = "/data/midas/proc"

ModuleNotFoundError: No module named 'rawio'

In [ ]:
#Select a file.
series='48873'
csv = [x[0] for x in pd.read_csv('~/git/R76Firmware/DCRC_Compatibility/manual_48873.txt',dtype=str,comment='#').values.tolist()]
#txt = [x[0] for x in pd.read_csv('~/git/R76Firmware/DCRC_Compatibility/towertest.txt',dtype=str).values.tolist()]

#More setup, dependent on series number.
procseries=series
filepath = rawdir+series

events = io.getRawEvents(rawdir,series)
try:
    stamps = [f'{QI[0]:04X}' for QI in events['Z6']['QI'][events['Z6']['PA'].keys()[0][0]]]
except KeyError:
    print(events.keys(),len(events))

del events

print(csv['timestamp'],[f'{i:x}' for i in stamps])
print(type(csv['timestamp'][0]),type(f'{100:x}'))
print('1337' in csv['timestamp'].values)

In [ ]:
print(stamps,"\n====\n",csv,len(stamps),len(csv))

In [ ]:
shared = [x for x in csv if x in stamps]
print(shared)
csvshared = csv[csv.index(shared[0]):]#csv.index(shared[-1])+1]
stampshared = stamps[stamps.index(shared[0]):]#stamps.index(shared[-1])+1]
#stampshared = [x for x in stampshared if x != '014A']

In [ ]:
print(csvshared,stampshared)

In [ ]:
for x in list(zip(csvshared,stampshared)): 
    if(x[0][:2] == x[1][:2]): print(x,"*")
    else: print(x)
print(shared)

In [ ]:
print(len(stamps)-len(set(stamps)))
print(stamps)

In [ ]:
repeats = [x for x in stamps if stamps.count(x) > 1]
print(set(repeats), (len(repeats)-len(set(repeats)))/len(set(repeats)))

for x in stamps:
    print(x+(stamps.count(x)-1)*"*")

So it seems like none of these are aligned. I'm switching to using a FIFO instead of RAM so that I can store a handful of values -- maybe it's getting overwritten by one of the next few before it has a chance to write to the packet, so a different value is ultimately saved.
- On next iteration, just set the FIFO to max size -- we actually need more space than we have now, since the rate could go much higher, and if it's too small then the whole system will fail. There's a limit to this, where if we can handle that trigger rate, we can handle anything above it (because of the inhibit function), but there's not really any reason to try to keep this small, is there?

In [ ]:
for i,x in enumerate(stamps[stamps.index(shared[0]):]):
    print("{0:016b}".format(int(x,16) ^ int(csv[csv.index(shared[0]):][i],16)))

In [ ]:
for i,x in enumerate(stamps[stamps.index(shared[0]):]):
    print(x == csv[csv.index(shared[0]):][i])

Upside: fewer obvious red flag values like beef or 1337. I think the repetition is a red flag. Is there anything in the code that tells the DCRC to retrieve the same value again, maybe?

In [ ]:
print(kill_dupes)

# Algorithm
from R76Analysis/example_scripts/time_difference...

In [ ]:
from itertools import zip_longest

bold_text = "\033[1m"
reset_format = "\033[0m"
title = "Merge and look for doubles:"
print(f"{bold_text}{title}{reset_format}")

merged_list = [item for pair in zip_longest(stamps,csv, fillvalue=None) for item in pair if item is not None]

for i,s in enumerate(merged_list):
    print(i,s)

trash=set()
i = 0
while i + 2 < len(merged_list):
    if merged_list[i] == merged_list[i + 1]:
        #print(":)")
        i += 2
    else:
        # Delete the single occurrence (lonely one)
        del merged_list[i]
        i += 1
            
for i in sorted(trash, reverse=True):
    del merged_list[i]

print()
print("New:")
for i,s in enumerate(merged_list):
    print(i,s)

todo:
- let it search the rest of the list, in case there are extras to be dropped
- tolerance +/- 1?